# Entrenamiento de audio con corrupciones
Hecho por:
* Ammi Beltrán
* Fernanda Borja
* Luciano Vidal

**Dirección del dataset**

In [1]:
DATA = "D:\\OneDrive\\OneDrive - Universidad de Chile\\Semestre X\\Teoria de la informacion\\proyecto\\UrbanSound8K\\UrbanSound8K"

## Librerías

In [2]:
# Misc
import os
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
# Locals
import audio.audio_corrupts as corrupt
import audio.audio_model as audiotrain
import audio.audio_data as dataudio
# Torch
import torch
import torchaudio
from torch import nn
from torch.utils.data import DataLoader

## Dataset

In [3]:
file = os.path.join(DATA, "metadata", "UrbanSound8K.csv")
df = pd.read_csv(file)
df['relative_path'] = '\\audio' + '\\fold' + df['fold'].astype(str) + '\\' + df['slice_file_name'].astype(str)
df = df[['relative_path', 'classID']]
df.head()

,relative_path,classID
0,\audio\fold5\100032-3-0-0.wav,3
1,\audio\fold5\100263-2-0-117.wav,2
2,\audio\fold5\100263-2-0-121.wav,2
3,\audio\fold5\100263-2-0-126.wav,2
4,\audio\fold5\100263-2-0-137.wav,2


In [4]:
len(df)

8732

In [5]:
from torch.utils.data import random_split
from sklearn.model_selection import train_test_split
num_items = len(df)
num_train = round(num_items * 0.8)
num_val = num_items - num_train
train, val = train_test_split(df, test_size=0.2,random_state=420)
train, val = train.reset_index(drop = True), val.reset_index(drop = True) 
# generator1 = torch.Generator().manual_seed(420)
# train, val = random_split(df, [num_train, num_val], generator=generator1)
# print(val)
train_ds = dataudio.SoundDS(train, DATA, transform = True)
val_ds = dataudio.SoundDS(val, DATA, transform = False)
# print(len(train_ds))
# print(val_ds[1])
#
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=250, shuffle=True)
val_dl = torch.utils.data.DataLoader(val_ds, batch_size=250, shuffle=False)

In [6]:
# from torch.utils.data import random_split

# myds = dataudio.SoundDS(df, DATA)

# # Random split of 80:20 between training and validation
# num_items = len(myds)
# num_train = round(num_items * 0.8)
# num_val = num_items - num_train
# generator1 = torch.Generator().manual_seed(420)
# train_ds, val_ds = random_split(myds, [num_train, num_val], generator=generator1)
# # Create training and validation data loaders
# train_dl = torch.utils.data.DataLoader(train_ds, batch_size=250, shuffle=True)
# val_dl = torch.utils.data.DataLoader(val_ds, batch_size=250, shuffle=False)

In [7]:
model = audiotrain.AudioModel()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
train = audiotrain.train(model, 100, train_dl, val_dl, criterion, optimizer, state = "train_params\\silent_37", name = "silent", dir = "train_params", device = "cuda")

Epoch nro 1/100
Iter: 1/28, Loss:2.303687572479248
Iter: 2/28, Loss:2.2573790550231934
Iter: 3/28, Loss:2.2285776138305664
Iter: 4/28, Loss:2.2107887268066406
Iter: 5/28, Loss:2.1722755432128906
Iter: 6/28, Loss:2.184400796890259
Iter: 7/28, Loss:2.1325724124908447
Iter: 8/28, Loss:2.1007535457611084
Iter: 9/28, Loss:2.0792980194091797
Iter: 10/28, Loss:2.0877773761749268
Iter: 11/28, Loss:2.0379438400268555
Iter: 12/28, Loss:2.0695457458496094
Iter: 13/28, Loss:2.0393948554992676
Iter: 14/28, Loss:2.0206034183502197
Iter: 15/28, Loss:1.9525399208068848
Iter: 16/28, Loss:2.0336592197418213
Iter: 17/28, Loss:1.9513278007507324
Iter: 18/28, Loss:1.9790844917297363
Iter: 19/28, Loss:1.958447813987732
Iter: 20/28, Loss:1.9477356672286987
Iter: 21/28, Loss:1.933546781539917
Iter: 22/28, Loss:1.9193873405456543
Iter: 23/28, Loss:1.9255808591842651
Iter: 24/28, Loss:1.8758074045181274
Iter: 25/28, Loss:1.872607707977295
Iter: 26/28, Loss:1.9030793905258179
Iter: 27/28, Loss:1.89034104347229
I

KeyboardInterrupt: 

: 